<a href="https://colab.research.google.com/github/n3xtss/CourseWork/blob/main/Section_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary packages
install.packages("sqldf")
install.packages("dplyr")
install.packages("ggplot2")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘plogr’, ‘gsubfn’, ‘proto’, ‘RSQLite’, ‘chron’




In [2]:
# Load libraries
library(sqldf)
library(dplyr)
library(ggplot2)

ERROR: Error in library(sqldf): there is no package called ‘sqldf’


In [27]:
#Step 1; Load the dataset from github

# Read CSV from GitHub
url <- "https://raw.githubusercontent.com/n3xtss/CourseWork/refs/heads/main/greenfuture_ideas_dataset.csv"
data <- read.csv(url)

# Preview
head(data)

,Employee.ID,Office.Location,Department,Idea.Submission.Date,Idea.Title,Idea.Category,Number.of.Votes,Collaboration.Status,Approval.Status,Implementation.Status,Security.Concern.Flag
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>
1,587b45d8,Germany,Renewable Energy,16/04/2024,Idea 1,Sustainable Transport,231,Single Contributor,Pending,Not Started,No
2,0d3e8b8e,Canada,Renewable Energy,14/01/2024,Idea 2,Water Management,477,Cross-Regional Team,Approved,In Progress,No
3,fa012fe7,France,Environmental Policy,10/01/2025,Idea 3,Renewable Energy,48,Cross-Regional Team,Approved,Not Started,Yes
4,cb8ed680,Japan,Technology,24/12/2023,Idea 4,Circular Economy,330,Cross-Regional Team,Approved,Completed,No
5,333fff26,Australia,Technology,13/02/2025,Idea 5,Circular Economy,428,Single Contributor,Approved,Not Started,No
6,a34ebb0f,India,Technology,08/09/2024,Idea 6,Circular Economy,431,Single Contributor,Rejected,Not Started,No


In [56]:
#Step 2; SQL queries

# Query 1: Select approved ideas that are not a security concern and have already started
approved <- sqldf("SELECT * FROM ideas
                  WHERE `Approval.Status` = 'Approved' AND `Security.Concern.Flag` = 'No' AND `Implementation.Status` != 'Not Started'
                  ORDER BY `Number.of.Votes` DESC")


# Query 2: Get top 5 ideas with most votes that have been approved and order by the number of votes
top5ideas <- sqldf("SELECT * FROM ideas
                    WHERE `Office.Location` != 'China' AND `Approval.Status` = 'Approved'
                    ORDER BY `Number.of.Votes` DESC ")

head(approved)
head(top5ideas)

,Employee.ID,Office.Location,Department,Idea.Submission.Date,Idea.Title,Idea.Category,Number.of.Votes,Collaboration.Status,Approval.Status,Implementation.Status,Security.Concern.Flag
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>
1,de5ed43f,USA,Technology,09/03/2024,Idea 2310,Renewable Energy,499,Cross-Regional Team,Approved,In Progress,No
2,0509f995,South Africa,Urban Development,15/10/2023,Idea 1810,Smart Cities,497,Single Contributor,Approved,In Progress,No
3,4f81ff0e,China,Infrastructure,10/01/2024,Idea 2559,Smart Cities,497,Single Contributor,Approved,In Progress,No
4,a3790157,Australia,Urban Development,09/04/2023,Idea 2824,Circular Economy,497,Single Contributor,Approved,In Progress,No
5,5aca5661,South Africa,Technology,27/05/2024,Idea 924,Sustainable Transport,496,Single Contributor,Approved,In Progress,No
6,1a4c4fc6,UK,Technology,13/05/2024,Idea 1984,Circular Economy,496,Cross-Regional Team,Approved,Completed,No


,Employee.ID,Office.Location,Department,Idea.Submission.Date,Idea.Title,Idea.Category,Number.of.Votes,Collaboration.Status,Approval.Status,Implementation.Status,Security.Concern.Flag
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>
1,7d239d23,USA,Urban Development,18/10/2024,Idea 1020,Circular Economy,500,Cross-Regional Team,Approved,In Progress,Yes
2,de5ed43f,USA,Technology,09/03/2024,Idea 2310,Renewable Energy,499,Cross-Regional Team,Approved,In Progress,No
3,4b644995,USA,Renewable Energy,04/06/2024,Idea 1876,Renewable Energy,498,Single Contributor,Approved,Not Started,No
4,0509f995,South Africa,Urban Development,15/10/2023,Idea 1810,Smart Cities,497,Single Contributor,Approved,In Progress,No
5,a3790157,Australia,Urban Development,09/04/2023,Idea 2824,Circular Economy,497,Single Contributor,Approved,In Progress,No
6,634c4f72,South Africa,Technology,26/07/2023,Idea 761,Water Management,496,Cross-Regional Team,Approved,Not Started,No


In [41]:
#Step 3; Aggregate Functions

# Count ideas per department
ideasDept <- sqldf("SELECT Department, COUNT(*) AS Total_Ideas
                    FROM data
                    GROUP BY Department
                    ORDER by Total_Ideas DESC")

# Average votes by office
avgVotesOffice <- sqldf("SELECT `Office.Location`, AVG(`Number.of.Votes`) AS Avg_Votes
                         FROM ideas
                         GROUP BY `Office.Location`
                         ORDER BY Avg_Votes DESC")




# Max votes by category
maxVotesCat <- sqldf("SELECT `Idea.Category`, MAX(`Number.of.Votes`) AS Max_Votes
                      FROM ideas
                      GROUP BY `Idea.Category`
                      ORDER BY Max_Votes DESC")

head(ideasDept)
head(avgVotesOffice)
head(maxVotesCat)


,Department,Total_Ideas
,<chr>,<int>
1,Infrastructure,547
2,Renewable Energy,513
3,Environmental Policy,499
4,Research,498
5,Urban Development,486
6,Technology,457


,Office.Location,Avg_Votes
,<chr>,<dbl>
1,Mexico,264.7542
2,Japan,263.2432
3,Brazil,254.9207
4,Australia,254.5205
5,USA,250.7801
6,China,248.9714


,Idea.Category,Max_Votes
,<chr>,<int>
1,Water Management,500
2,Sustainable Transport,500
3,Renewable Energy,500
4,Circular Economy,500
5,Carbon Capture,499
6,Smart Cities,498


In [1]:
#Step 4; Business Analysis

# Find departments with most approved ideas
deptVotes <- sqldf("SELECT Department, COUNT(*) AS Approved_Ideas FROM ideas
                    WHERE `Approval.Status` = 'Approved' GROUP BY Department
                    ORDER BY Approved_Ideas DESC")

# Find top collaborating regions by number of 'Cross-Regional Team' submissions
topCollab <- sqldf("SELECT `Office.Location`, COUNT(*) AS Team_Ideas FROM ideas
                    WHERE `Collaboration.Status` = 'Cross-Regional Team'
                    GROUP BY `Office.Location`
                    ORDER BY Team_Ideas DESC")

head(deptVotes)
head(topCollab)

ERROR: Error in sqldf("SELECT Department, COUNT(*) AS Approved_Ideas FROM ideas \n                    WHERE `Approval.Status` = 'Approved' GROUP BY Department \n                    ORDER BY Approved_Ideas DESC"): could not find function "sqldf"
